In [1]:
#-*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score,log_loss, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
pd.options.display.max_columns=50
pd.options.display.max_rows=20
import warnings
warnings.filterwarnings('ignore')

In [2]:
def run_lgb_cv(train_X,train_Y,lgb_params,feature_name=None,split=5,seed=6666,cat_list=None, threshold = 0.5):
    val_results = []
    models_list = []
    best_iterations = []
    train_pred = np.zeros(train_X.shape[0])
    

    # recall 和precision 的score
    def lgb_self_score(y_pred, data):
        y_true = data.get_label()  #lgb_data获取label
        # y_pred = np.round(y_pred) # 默认精度为0，四舍五入
        y_pred = np.where(y_pred>=threshold,1,0)
        return [('accuracy_score', accuracy_score(y_true, y_pred), True),('precision_score', precision_score(y_true, y_pred), True),('recall_score', recall_score(y_true, y_pred), True)]

        

    seeds=range(seed,seed+split)
    feature_importance_df = pd.DataFrame()
    
    if feature_name == None:
        feature_name = [col for col in train_X.columns if col!='index' and col!='id' and col!='isNew']
    print('Using features:',feature_name)
    
    
    # 分类器GroupKFold
    gkf = GroupKFold(n_splits=split)
    index = 0
    for train_index, test_index in gkf.split(train_X, train_Y, groups=train_X['ID']):
        
    # for index, (train_index, test_index) in enumerate(train_val_spliter.split(train_X, train_Y)):
        print('fold:',index+1)
        lgb_params['random_state'] = seeds[index]
        index += 1 
        
        val_result = []
        #划分训练集及标签，验证集及标签
        train_x, val_x, train_y, val_y = train_X[feature_name].iloc[train_index], train_X[feature_name].iloc[test_index], train_Y.iloc[train_index], train_Y.iloc[test_index]
        
        # 标签0与标签1的比
        neg_pos_ratio = np.log(len(train_y[train_y==0])/len(train_y[train_y==1]))
        #neg_pos_ratio = 1
        print('neg_pos_ratio:',neg_pos_ratio)
        
        # 使用lgb的必须操作：转化为lgb的dataset
        train_data = lgb.Dataset(train_x,label=train_y)
        val_data = lgb.Dataset(val_x,label=val_y)
        # 种类特征的相关处理
        if cat_list == None:
            cat_list = ""
        #lgb_model = lgb.LGBMClassifier(**lgb_params)
        # 进行训练
        metric_dict = {}
        lgb_model=lgb.train(lgb_params,train_data,valid_sets=[val_data],verbose_eval=lgb_params['verbose'],
                            callbacks = [lgb.record_evaluation(metric_dict)], feval=lgb_self_score)
        #lgb_model.fit(train_x,train_y,eval_set=(val_x,val_y))
        gc.collect() # 清内存
        
        # 把验证集预测结果按照test_index加到predict矩阵中，这样数折验证后刚好填满train_pred矩阵
        train_pred[test_index] += lgb_model.predict(val_x[feature_name])
        
        # 生成有feature和importance的特征重要度矩阵
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = pd.Series(feature_name)
        fold_importance_df["importance"] = pd.Series(lgb_model.feature_importance(importance_type='gain'))
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        # 计算分数
        val_result.append(accuracy_score(val_y, np.where(train_pred[test_index]>=threshold,1,0)))
        print('accuracy_score: ',val_result[-1])
        val_result.append(precision_score(val_y, np.where(train_pred[test_index]>=threshold,1,0)))
        print('precision_score: ',val_result[-1])
        val_result.append(recall_score(val_y, np.where(train_pred[test_index]>=threshold,1,0)))
        print('recall_score: ',val_result[-1])
        
        # 记录数据
        best_iterations.append(lgb_model.current_iteration())
        models_list.append(lgb_model)
        val_results.append(val_result)
    

    
    # 打印特征重要度
    # best_features返回模型中平均重要度的降序排列
    best_features = (feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False))
    best_features['feature'] = best_features.index  # 因为这样操作后index是特征名，需要修改
    best_features.reset_index(drop=True)
    plt.figure(figsize=(14,20))
    sns.barplot(x="importance", y="feature", data=best_features)
    plt.title('LightGBM Features (averaged over folds)')
    plt.tight_layout()

    val_results = np.array(val_results)
    print('cv completed')
    print('mean best iteration: ',np.mean(best_iterations))
    print('std best iteration: ',np.std(best_iterations))   # 标准差
    print('-'*40)
    train_pred = np.where(train_pred>=threshold,1,0)
    # 各折分数标准差，标准差越低越好
    dic = [{'平均迭代轮数 ':np.mean(best_iterations), '精确率方差':np.std(val_results[:,1]),'召回率方差':np.std(val_results[:,2]),
            '准确度率分数':accuracy_score(train_Y,train_pred),'精确率分数':precision_score(train_Y,train_pred),
            '召回率分数':recall_score(train_Y, train_pred),
            '分数综合':(accuracy_score(train_Y,train_pred)+precision_score(train_Y,train_pred)+recall_score(train_Y, train_pred))/3},
           {'平均迭代轮数 ':np.mean(best_iterations), '精确率方差':np.std(val_results[:,1]),'召回率方差':np.std(val_results[:,2]),
            '准确度率分数':accuracy_score(train_Y,train_pred),'精确率分数':precision_score(train_Y,train_pred),
            '召回率分数':recall_score(train_Y, train_pred),
            '分数综合':(accuracy_score(train_Y,train_pred)+precision_score(train_Y,train_pred)+recall_score(train_Y, train_pred))/3}]
    score_result = pd.DataFrame(dic,index = [0,1])
    #print('std accuracy_score: ',np.std(val_results[:,0]))
    #print('std precision_score: ',np.std(val_results[:,1]))
    #print('std recall_score: ',np.std(val_results[:,2]))
    # 最终分数，分数越高越好
    #print('oof accuracy_score：', accuracy_score(train_Y,train_pred))   # 准确率 （TP+TN）/（TP+TN+FN+FP）
    #print('oof precision_score：', precision_score(train_Y,train_pred))  # 精确率  TP  / (TP+ FP)  预测为正的样本中有多少为真
    #print('oof recall_score：', recall_score(train_Y, train_pred))    # 召回率  TP / (TP + FN)   结果为正的样本中有多少被预测正确了
    
    return train_pred,models_list,pd.DataFrame(val_results,columns=['accuracy_score','precision_score', 'recall_score']),best_features.sort_values(by="importance",ascending=False),score_result

In [3]:
# 训练集读取，45146*30，其中有标签的15215*30
train_data = pd.read_csv('base_train_sum.csv', encoding = 'gbk')
train_data = pd.merge(train_data,pd.read_csv('knowledge_train_sum.csv', encoding = 'gbk'),how='left',on='ID')
train_data = pd.merge(train_data,pd.read_csv('money_report_train_sum.csv', encoding = 'gbk'),how='left',on='ID')
train_data = pd.merge(train_data,pd.read_csv('year_report_train_sum.csv', encoding = 'gbk'),how='left',on=['ID','year']) # 这样就能一起合并
train_data = train_data[~train_data['flag'].isnull()].reset_index(drop=True)

# 验证集读取，96250*31，其中有标签的91732*30（注：比训练集多了一个控制人ID，官方回复测试集中不会加入这个特征，所以选择删除）
valid_data = pd.read_csv('base_verify1.csv', encoding = 'gbk')
valid_data = pd.merge(valid_data,pd.read_csv('paient_information_verify1.csv', encoding = 'gbk'),how='left',on='ID')
valid_data = pd.merge(valid_data,pd.read_csv('money_information_verify1.csv', encoding = 'gbk'),how='left',on='ID')
valid_data = pd.merge(valid_data,pd.read_csv('year_report_verify1.csv', encoding = 'gbk'),how='left',on=['ID','year'])
del valid_data['控制人ID']

# 合并为一个数据集，137796*30
data = pd.concat([train_data,valid_data],axis=0,sort=False,ignore_index=True)
data.columns = ['ID','Registration Time', 'Registered Capital', 'Industry', 'Region', 'Business Type', 'Controlling Type', 'Controlling Shareholding','flag', 'Patent', 'Trademark', 'Copyright', 'year', 'Debt financing line', 'Debt financing cost', 'Equity financing line', 'Equity financing cost', 'Internal financing and trade financing line', 'Internal financing and trade financing cost', 'Project financing and Policy financing quota ',' Project financing and policy financing costs', 'Number of employees','Total assets', 'Total liabilities',' Total operating income ',' Main operating income ',' Total profit ' , 'Net Profit', 'Total Taxes', 'Total Owners Equity']

# 去flag=null的值，现在data:106947*30
data = data[~data['flag'].isnull()].reset_index(drop=True)
# 打乱数据
# data = data.sample(frac=1).reset_index(drop=True)
# 去重
data = data.drop_duplicates(subset=None,keep='first',inplace=False)
# 所有nan值换0,不要使用，降低效果
# data = data.fillna(0)

#labelencode
#['Industry', 'Region', 'Business Type', 'Controlling Type']
#['行业','区域','企业类型','控制人类型']
for col in ['Industry', 'Region', 'Business Type', 'Controlling Type']:
    lbl = LabelEncoder()
    data[col] = lbl.fit_transform(data[col].astype(str))

In [4]:
# 特征工程 ——自身特征
data['Total Taxes Square'] = data['Total Taxes'] **2
data['log Total Taxes'] = np.log(data['Total Taxes'])
data['log Net Profit'] = np.log(data['Net Profit'])
                                
# qcut等频分箱，cut为等距分箱
cats = pd.qcut(data['Total Taxes'], 6, duplicates = 'drop')
data['Total Taxes qcut_branch6'] = lbl.fit_transform(cats.astype(str))
cats = pd.qcut(data['Net Profit'], 8, duplicates = 'drop')
data['Net Profit qcut_branch8'] = lbl.fit_transform(cats.astype(str))


In [5]:
#特征工程2
datag = data.groupby('ID')
lst = list(set(data['ID']))
Total_Taxes_incre = []         
Total_Taxes_incre_mean = []
Net_profit_has_0 = []
Total_Taxes_has_0 = []
for id in lst:
    group = datag.get_group(id)
    taxes = list(group['Total Taxes'])
    nets = list(group['Net Profit'])
    
    # 判断三年利润为0/有1~2年为0/都存在值
    if sum(nets)==0:  
        Net_profit_has_0.extend([0]*len(nets))
    elif(0 in nets):
        Net_profit_has_0.extend([1]*len(nets)) 
    else:
        Net_profit_has_0.extend([2]*len(nets)) 
    
    # 判断三年税收为0/有1~2年为0/都存在值
    if sum(taxes)==0:
        Total_Taxes_has_0.extend([0]*len(taxes))
    elif(0 in taxes):
        Total_Taxes_has_0.extend([1]*len(taxes)) 
    else:
        Total_Taxes_has_0.extend([2]*len(taxes)) 
    
    # 计算总税收增长率
    if 0 in taxes or len(taxes)<3:
        Total_Taxes_incre_mean.extend([0]*len(taxes))
    else:
        tax_mean = (taxes[1]/taxes[0] + taxes[2]/taxes[1] )/2
        Total_Taxes_incre_mean.extend([tax_mean]*len(taxes))
        
data['Net_profit_has_0'] = Net_profit_has_0
data['Total_Taxes_has_0'] = Total_Taxes_has_0  
data['Total_Taxes_incre_mean'] = Total_Taxes_incre_mean


In [6]:
# 特征工程3
# data['asset < liabilities'] = data['Total assets'] < data['Total liabilities']#资不抵债
data['year*Net Profit'] = data['year'] * data['Net Profit']#搜索特征
data['Industry*Total Taxes'] = data['Industry'] * data['Total Taxes'] # 搜索特征
data['Asset liability ratio'] = data['Total assets']/data['Total liabilities'] #资产负债比率

In [7]:
# 聚类
data1 = data.fillna(0)
data1 = data1.replace([np.inf, -np.inf],0) #替换正负inf为NA
estimator = KMeans(n_clusters=6, n_jobs = 8) #构造聚类器
estimator.fit(data1)
label_pred = estimator.labels_#最终聚类类别  

data['Kmeans_label'] = label_pred  # 聚类结果作为一维特征

In [8]:
# 获取特征名
feature_name = [col for col in data.columns if col not in ['ID','flag']]
print(feature_name)

['Registration Time', 'Registered Capital', 'Industry', 'Region', 'Business Type', 'Controlling Type', 'Controlling Shareholding', 'Patent', 'Trademark', 'Copyright', 'year', 'Debt financing line', 'Debt financing cost', 'Equity financing line', 'Equity financing cost', 'Internal financing and trade financing line', 'Internal financing and trade financing cost', 'Project financing and Policy financing quota ', ' Project financing and policy financing costs', 'Number of employees', 'Total assets', 'Total liabilities', ' Total operating income ', ' Main operating income ', ' Total profit ', 'Net Profit', 'Total Taxes', 'Total Owners Equity', 'Total Taxes Square', 'log Total Taxes', 'log Net Profit', 'Total Taxes qcut_branch6', 'Net Profit qcut_branch8', 'Net_profit_has_0', 'Total_Taxes_has_0', 'Total_Taxes_incre_mean', 'year*Net Profit', 'Industry*Total Taxes', 'Asset liability ratio', 'Kmeans_label']


In [9]:
data

,ID,Registration Time,Registered Capital,Industry,Region,Business Type,Controlling Type,Controlling Shareholding,flag,Patent,Trademark,Copyright,year,Debt financing line,Debt financing cost,Equity financing line,Equity financing cost,Internal financing and trade financing line,Internal financing and trade financing cost,Project financing and Policy financing quota,Project financing and policy financing costs,Number of employees,Total assets,Total liabilities,Total operating income,Main operating income,Total profit,Net Profit,Total Taxes,Total Owners Equity,Total Taxes Square,log Total Taxes,log Net Profit,Total Taxes qcut_branch6,Net Profit qcut_branch8,Net_profit_has_0,Total_Taxes_has_0,Total_Taxes_incre_mean,year*Net Profit,Industry*Total Taxes,Asset liability ratio,Kmeans_label
0,28,2007.0,2050.0,1,7,1,1,NaN,1.0,0.0,1.0,1.0,2015.0,0.0,0.00,0.0,0.000,21648.0,1298.880,0.0,0.000,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,-7216.0,0.0,-12300.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,-14540240.0,0.0,0.571429,1
1,28,2007.0,2050.0,1,7,1,1,NaN,1.0,0.0,1.0,1.0,2016.0,0.0,0.00,34686.0,1387.440,0.0,0.000,0.0,0.000,396.0,73800.0,71750.0,346860.0,173430.0,173430.0,-34686.0,0.0,2050.0,0.000000e+00,-inf,NaN,0,2,2,0,0.0,-69926976.0,0.0,1.028571,1
2,28,2007.0,2050.0,1,7,1,1,NaN,1.0,0.0,1.0,1.0,2017.0,0.0,0.00,3444.0,137.760,0.0,0.000,0.0,0.000,393.0,82000.0,159900.0,172200.0,103320.0,17220.0,-17220.0,0.0,-77900.0,0.000000e+00,-inf,NaN,0,2,2,0,0.0,-34732740.0,0.0,0.512821,1
3,230,2008.0,3360.0,4,2,1,1,1.00,1.0,0.0,0.0,0.0,2015.0,0.0,0.00,0.0,0.000,0.0,0.000,470.4,28.224,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,-11524.8,0.0,13440.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,-23222472.0,0.0,2.333333,1
4,230,2008.0,3360.0,4,2,1,1,1.00,1.0,0.0,0.0,0.0,2016.0,0.0,0.00,0.0,0.000,46771.2,2806.272,0.0,0.000,365.0,53760.0,50400.0,155904.0,124723.2,46771.2,-15590.4,0.0,3360.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,-31430246.4,0.0,1.066667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106942,5999998,2014.0,4510.0,4,6,4,2,0.64,0.0,0.0,0.0,0.0,2017.0,0.0,0.00,0.0,0.000,18265.5,1095.930,0.0,0.000,841.0,67650.0,63140.0,60885.0,30442.5,30442.5,-6088.5,0.0,4510.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,-12280504.5,0.0,1.071429,1
106943,5999999,2014.0,9130.0,1,7,4,2,0.80,0.0,1.0,1.0,1.0,2015.0,0.0,0.00,6025.8,241.032,0.0,0.000,0.0,0.000,885.0,27390.0,9130.0,60258.0,42180.6,30129.0,6025.8,24103.2,18260.0,5.809643e+08,10.090100,8.703806,1,6,2,0,0.0,12141987.0,24103.2,3.000000,1
106944,5999999,2014.0,9130.0,1,7,4,2,0.80,0.0,1.0,1.0,1.0,2016.0,7304.0,584.32,0.0,0.000,0.0,0.000,0.0,0.000,933.0,73040.0,95865.0,51128.0,25564.0,5112.8,0.0,10225.6,-22825.0,1.045629e+08,9.232650,-inf,1,3,2,0,0.0,0.0,10225.6,0.761905,1
106945,5999999,2014.0,9130.0,1,7,4,2,0.80,0.0,1.0,1.0,1.0,2017.0,0.0,0.00,0.0,0.000,0.0,0.000,821.7,49.302,46.0,82170.0,73040.0,16434.0,11503.8,8217.0,6573.6,9860.4,9130.0,9.722749e+07,9.196282,8.790817,1,6,2,0,0.0,13258951.2,9860.4,1.125000,1


In [10]:
# baseline
params1 = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': ['accuracy_score','recall_score','precision_score'],
            'first_metric_only':True,
            'learning_rate': 0.05,
            'max_depth': 9,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 20,
            'reg_alpha': 3.77, 
            'reg_lambda': 5.86,
            'num_boost_round':1000,
            'verbose':50,
            'early_stopping_rounds':100
        }

#hyperopt自动调参（准确率核心）
params2 = {'boosting_type': 'gbdt',
           'objective': 'binary',
           'bagging_freq': 0,
            'metric': ['accuracy_score','recall_score','precision_score'],
            'first_metric_only':True,
            'learning_rate': 0.06264998043144464, 
           'max_depth': 2,
            'num_leaves': 2,
            'feature_fraction': 0.531632694197028,
            'bagging_fraction': 0.8863315903648026, #平衡数据抽样
           'reg_alpha': 8.786296231756943,
            'reg_lambda': 1.5592939193176238,
            'num_boost_round':1000,
            'verbose':50,
            'early_stopping_rounds':100
          }

# hyperopt自动调参（重视准确率、精确率、召回率）
params3 = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'bagging_freq': 0,
    'metric': ['accuracy_score','recall_score','precision_score'],
    'first_metric_only':True,
    'bagging_freq': 9,
    
     'feature_fraction': 0.5977477423637545,
     'learning_rate': 0.06475934567788924,
     'max_depth': 2,
     'bagging_fraction': 0.8988155155440889,
     'num_leaves': 2,
     'num_trees': 289,
     'reg_alpha': 2.4691423840573465,
     'reg_lambda': 0.5276467812362184,
    'num_boost_round':1000,
    'verbose':50,
    'early_stopping_rounds':100
}

# 手动1
params4 = {
    'boosting_type':'gbdt','objective':'binary', 'metric': ['accuracy_score','recall_score','precision_score'],'first_metric_only':True,
    'learning_rate':0.01, 'n_estimators':267, 'max_depth':4,
    'num_leaves':5,'max_bin':35,'min_data_in_leaf':31,'bagging_fraction':0.6,'bagging_freq':0,
    'feature_fraction': 0.7, 'lambda_l1':0.5,'lambda_l2':0.001, 'verbose':50, 'early_stopping_rounds':100
}

# 手动随缘
params5 = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': ['accuracy_score','recall_score','precision_score'],
            'first_metric_only':True,
            'learning_rate': 0.01,
            'max_depth': 9,
            'num_leaves': 63,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 20,
            'reg_alpha': 3.77, 
            'reg_lambda': 5.86,
            'num_boost_round':1000,
            'verbose':50,
            'early_stopping_rounds':100
        }

# 手动准确率调参（最终选取结果）
params6 = {
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':['accuracy_score', 'precision_score', 'recall_score'],
    'first_metric_only':True,
    'learning_rate':0.05,
    'n_estimators': 54,
    'max_depth':4,
    'num_leaves' : 10,
    'bagging_fraction' : 0.9,
    'feature_fraction' : 1.0,
    'max_bin' : 255,
    'min_data_in_leaf' : 71,
    'bagging_freq' : 40,
    'lambda_l1' : 0.1,
    'lambda_l2' : 0.9,
    'min_split_gain' : 0.4,
    'num_boost_round':1000,
    'verbose':50,
    'early_stopping_rounds':100,
}


In [ ]:
# 投入data进行训练
%time oof,model_list,val_result_df,importance, score_result = run_lgb_cv(data,data['flag'].astype(int),params6,feature_name,seed = 6666, split=5,threshold = 0.46)

Using features: ['Registration Time', 'Registered Capital', 'Industry', 'Region', 'Business Type', 'Controlling Type', 'Controlling Shareholding', 'Patent', 'Trademark', 'Copyright', 'year', 'Debt financing line', 'Debt financing cost', 'Equity financing line', 'Equity financing cost', 'Internal financing and trade financing line', 'Internal financing and trade financing cost', 'Project financing and Policy financing quota ', ' Project financing and policy financing costs', 'Number of employees', 'Total assets', 'Total liabilities', ' Total operating income ', ' Main operating income ', ' Total profit ', 'Net Profit', 'Total Taxes', 'Total Owners Equity', 'Total Taxes Square', 'log Total Taxes', 'log Net Profit', 'Total Taxes qcut_branch6', 'Net Profit qcut_branch8', 'Net_profit_has_0', 'Total_Taxes_has_0', 'Total_Taxes_incre_mean', 'year*Net Profit', 'Industry*Total Taxes', 'Asset liability ratio', 'Kmeans_label']
fold: 1
neg_pos_ratio: 0.43593939529305703
Training until validation sc

In [ ]:
score_result#6

In [ ]:
# 分别对应文件夹名，四个csv的名称必须按顺序传入
def get_test_data(folder, base_train_sum, knowledge_train_sum, money_report_train_sum, year_report_train_sum):
    '''
    author： 叶文涛
    time：2020-7-15
    功能：读入测试集，调用模型进行预测
    参数说明——
    folder： 文件夹名称
    base_test_sum： 测试集1
    knowledge_test_sum： 测试集2
    money_report_test_sum： 测试集3
    year_report_test_sum： 测试集4
    '''
    # 训练集读取
    data = pd.read_csv(folder + base_train_sum)
    data = pd.merge(data,pd.read_csv(folder + knowledge_train_sum),how='left',on='ID')
    data = pd.merge(data,pd.read_csv(folder + money_report_train_sum),how='left',on='ID')
    data = pd.merge(data,pd.read_csv(folder + year_report_train_sum),how='left',on=['ID','year']) # 这样就能一起合并
    
    # 如果有控制人ID，flag则删除
    try:del data['flag']
    except:pass
    try:del data['控制人ID']
    except:pass
        
    data.columns =  ['ID','Registration Time', 'Registered Capital', 'Industry', 'Region', 'Business Type',
                     'Controlling Type', 'Controlling Shareholding', 'Patent', 'Trademark', 'Copyright',
                     'year', 'Debt financing line', 'Debt financing cost', 'Equity financing line',
                     'Equity financing cost', 'Internal financing and trade financing line',
                     'Internal financing and trade financing cost', 'Project financing and Policy financing quota ',
                     ' Project financing and policy financing costs', 'Number of employees','Total assets',
                     'Total liabilities',' Total operating income ',' Main operating income ',' Total profit ' ,
                     'Net Profit', 'Total Taxes', 'Total Owners Equity']


    #labelencode
    for col in ['Industry', 'Region', 'Business Type', 'Controlling Type']:
        lbl = LabelEncoder()
        data[col] = lbl.fit_transform(data[col].astype(str))
        
    # 特征工程
    data['Total Taxes Square'] = data['Total Taxes'] **2
    data['log Total Taxes'] = np.log(data['Total Taxes'])
    data['log Net Profit'] = np.log(data['Net Profit'])
    # qcut等频分箱，cut为等距分箱
    cats = pd.qcut(data['Total Taxes'], 6, duplicates = 'drop')
    data['Total Taxes qcut_branch6'] = lbl.fit_transform(cats.astype(str))
    cats = pd.qcut(data['Net Profit'], 8, duplicates = 'drop')
    data['Net Profit qcut_branch8'] = lbl.fit_transform(cats.astype(str))
    #特征工程2
    datag = data.groupby('ID')
    lst = list(set(data['ID']))
    Total_Taxes_incre = []
    Total_Taxes_incre_mean = []
    Net_profit_has_0 = []
    Total_Taxes_has_0 = []
    for id in lst:
        group = datag.get_group(id)
        taxes = list(group['Total Taxes'])
        nets = list(group['Net Profit'])
        # 判断三年利润为0/有1~2年为0/都存在值
        if sum(nets)==0:  
            Net_profit_has_0.extend([0]*len(nets))
        elif(0 in nets):
            Net_profit_has_0.extend([1]*len(nets)) 
        else:
            Net_profit_has_0.extend([2]*len(nets)) 
        # 判断三年税收为0/有1~2年为0/都存在值
        if sum(taxes)==0:
            Total_Taxes_has_0.extend([0]*len(taxes))
        elif(0 in taxes):
            Total_Taxes_has_0.extend([1]*len(taxes)) 
        else:
            Total_Taxes_has_0.extend([2]*len(taxes)) 
        # 计算总税收增长率
        if 0 in taxes or len(taxes)<3:
            Total_Taxes_incre_mean.extend([0]*len(taxes))
        else:
            a = taxes[1]/taxes[0]
            b = taxes[2]/taxes[1]
            tax_mean = (a + b)/2
            Total_Taxes_incre_mean.extend([tax_mean]*len(taxes))
    data['Net_profit_has_0'] = Net_profit_has_0
    data['Total_Taxes_has_0'] = Total_Taxes_has_0  
    data['Total_Taxes_incre_mean'] = Total_Taxes_incre_mean
    # 特征工程3
    data['year*Net Profit'] = data['year'] * data['Net Profit']#搜索得到特征
    data['Industry*Total Taxes'] = data['Industry'] * data['Total Taxes'] # 搜索得到特征
    data['Asset liability ratio'] = data['Total assets']/data['Total liabilities'] #资产负债比率
    # 聚类
    #data = data.T #使待分类样本格式正确
    data1 = data.fillna(0)
    data1 = data1.replace([np.inf, -np.inf],0) #替换正负inf为NA
    del data1['ID']
    estimator = KMeans(n_clusters=6, n_jobs = 8) #构造聚类器
    estimator.fit(data1)
    label_pred = estimator.labels_#最终聚类类别  
    data['Kmeans_label'] = label_pred
    
    return data

'''使用示例：'''
# 调用函数，注意末端的/要加上
folder = ''
test = get_test_data(folder, 'base_test_sum.csv', 'knowledge_test_sum.csv', 'money_report_test_sum.csv', 'year_report_test_sum.csv')

In [ ]:
# 查看训练集与测试集特征分布
plt.subplots(9,5,figsize=(15,30))
for idx,col in enumerate(test.columns):
    plt.subplot(9,5,idx+1)
    plt.title(col)
    sns.kdeplot(data[col],label='train')
    sns.kdeplot(test[col],label='test')
